In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_excel(r"C:\Desktop\predictiveAnalysis\data\raw\who_air_quality.xlsx", engine = "openpyxl")

In [5]:
df.head(5)

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,4_Eur,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,87.0,46.0,93.0,"Urban, Urban, Suburban",NaN,NaN,246146.0,"manual, manual, manual, manual",43.367900,-8.418571,1
1,4_Eur,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,96.0,88.0,95.0,"Urban, Urban, Suburban",NaN,NaN,247604.0,NaN,43.368033,-8.418233,1
2,4_Eur,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,98.0,71.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
3,4_Eur,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,98.0,98.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
4,4_Eur,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,97.0,97.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1


In [6]:
df.shape

(40098, 20)

In [7]:
df.columns

Index(['who_region', 'iso3', 'country_name', 'city', 'year', 'version',
       'pm10_concentration', 'pm25_concentration', 'no2_concentration',
       'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov', 'type_of_stations',
       'reference', 'web_link', 'population', 'population_source', 'latitude',
       'longitude', 'who_ms'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40098 entries, 0 to 40097
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   who_region          40098 non-null  object 
 1   iso3                40098 non-null  object 
 2   country_name        40098 non-null  object 
 3   city                40098 non-null  object 
 4   year                40095 non-null  float64
 5   version             40098 non-null  object 
 6   pm10_concentration  28672 non-null  float64
 7   pm25_concentration  21730 non-null  float64
 8   no2_concentration   26934 non-null  float64
 9   pm10_tempcov        22403 non-null  float64
 10  pm25_tempcov        16590 non-null  float64
 11  no2_tempcov         23402 non-null  float64
 12  type_of_stations    23331 non-null  object 
 13  reference           6652 non-null   object 
 14  web_link            1790 non-null   object 
 15  population          22433 non-null  float64
 16  popu

In [9]:
df.describe()

,year,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,population,latitude,longitude,who_ms
count,40095.000000,28672.000000,21730.000000,26934.000000,22403.000000,16590.000000,23402.000000,2.243300e+04,40098.000000,40098.000000,40098.000000
mean,2016.232672,30.016375,19.130923,18.861620,88.732134,84.124533,88.092214,7.571931e+05,40.024341,10.598472,0.998803
std,3.116678,29.548575,17.490027,25.845061,20.277271,26.819096,23.149637,2.123730e+06,16.309381,57.774989,0.034578
min,2010.000000,0.730000,1.000000,0.002000,0.000000,0.000000,0.000000,5.000000e+00,-53.158316,-159.366240,0.000000
25%,2014.000000,16.264750,8.603250,10.163250,90.000000,86.000000,92.000000,5.300600e+04,37.138800,-1.258900,1.000000
50%,2016.000000,21.335500,12.710500,16.700000,96.000000,96.000000,96.000000,1.637290e+05,43.822900,10.512933,1.000000
75%,2019.000000,30.515500,23.000000,24.911500,99.000000,99.000000,98.000000,5.900320e+05,49.163367,23.226400,1.000000
max,2022.000000,540.000000,436.440000,3670.314000,100.000000,100.000000,100.000000,3.739313e+07,69.677498,178.450000,1.000000


In [10]:
df.isnull().sum()

who_region                0
iso3                      0
country_name              0
city                      0
year                      3
version                   0
pm10_concentration    11426
pm25_concentration    18368
no2_concentration     13164
pm10_tempcov          17695
pm25_tempcov          23508
no2_tempcov           16696
type_of_stations      16767
reference             33446
web_link              38308
population            17665
population_source     21996
latitude                  0
longitude                 0
who_ms                    0
dtype: int64

In [11]:
df.duplicated().sum()

np.int64(0)

## Cleaning of data

In [12]:
df = df.drop(columns = ['version','reference', 'web_link', 'population_source'])

In [13]:
# spliiting the city 
df['city'] = df['city'].str.split('/').str[0]


In [14]:
# the year is in float so converting to int 
df['year'] = df['year'].astype('Int64')


In [15]:
# type_of_stations contains multiple values like citynae, country which is related to other col.. so splitting it 
df['type_of_stations'] = df['type_of_stations'].str.split(',').str[0].str.strip()


In [16]:
df['who_region'] = df['who_region'].str.split('_').str[1]


## handling missing values

In [17]:
df['year'] = df['year'].fillna(df['year'].bfill())

In [18]:
df['type_of_stations'] = df['type_of_stations'].bfill().ffill()


In [19]:
# skewness for checking asymmetry in data 

df[['pm10_concentration','pm25_concentration','no2_concentration','population']].skew()


pm10_concentration      4.361872
pm25_concentration      3.396063
no2_concentration     106.488175
population              7.405647
dtype: float64

In [20]:
pollution_cols = [
    'pm10_concentration', 'pm25_concentration', 'no2_concentration'
]
for col in pollution_cols:
    df[col] = df[col].fillna(df[col].median())

In [21]:
df['population'] = df['population'].fillna(df['population'].median())

In [22]:
df[['pm10_tempcov', 'pm25_tempcov', 'no2_tempcov']].skew()

pm10_tempcov   -2.930853
pm25_tempcov   -2.189910
no2_tempcov    -2.994921
dtype: float64

In [23]:
tempcov_cols = [
    'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov'
]
for col in tempcov_cols:
    df[col] = df[col].fillna(df[col].median())

## Outlier detection

In [24]:
num_cols = ['pm10_concentration','pm25_concentration','no2_concentration','pm10_tempcov',
                'pm25_tempcov','no2_tempcov','population']

In [25]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

In [26]:
df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

## Feature Engineering

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
df["pollution_index"] = pca.fit_transform(
    df[["pm10_concentration", "pm25_concentration", "no2_concentration"]]
)


In [28]:
df['pollution_per_person'] = (df['pm10_concentration']/df['population'] + 
                         df['pm25_concentration']/df['population'] + 
                         df['no2_concentration']) / df['population']


In [29]:
df = df.sort_values(by=['city', 'year'])

df['pollution_index_yearly_change'] = df.groupby('city')['pollution_index'].diff()
df['pollution_index_yearly_change'] = df['pollution_index_yearly_change'].fillna(0)

## Scaling

In [30]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

cols = ['pm10_concentration', 'pm25_concentration', 'no2_concentration',
        'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov', 'population', 'pollution_per_person', 'pollution_index']

scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])


## Encoding

In [31]:
ohe = OneHotEncoder(sparse_output=False)
le = LabelEncoder()

df['type_of_stations'] = le.fit_transform(df['type_of_stations'])

df['who_region'] = ohe.fit_transform(df[['who_region']])
df['country_name'] = ohe.fit_transform(df[['country_name']])
df['iso3'] = ohe.fit_transform(df[['iso3']])
df['city'] = ohe.fit_transform(df[['city']])


In [32]:
df.head()

,who_region,iso3,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,population,latitude,longitude,who_ms,pollution_index,pollution_per_person,pollution_index_yearly_change
0,0.0,0.0,0.0,1.0,2013,-0.170206,-0.354191,0.504059,-0.317035,-2.475293,0.089188,9,0.649804,43.367900,-8.418571,1,-0.095652,-0.041423,0.000000
1,0.0,0.0,0.0,1.0,2014,-0.002639,-0.023508,0.067105,0.260630,-0.169461,0.199629,9,0.663666,43.368033,-8.418233,1,0.008241,-0.042623,2.702171
2,0.0,0.0,0.0,1.0,2015,-0.080175,-0.164766,0.215932,0.389000,-1.102774,0.365289,9,0.663666,43.370375,-8.422900,1,-0.049060,-0.042222,-1.490347
3,0.0,0.0,0.0,1.0,2016,-0.177362,-0.228385,0.096767,0.389000,0.379546,0.365289,9,0.663666,43.370375,-8.422900,1,-0.168826,-0.042543,-3.115034
4,0.0,0.0,0.0,1.0,2017,-0.027430,-0.156475,0.159910,0.324815,0.324645,0.365289,9,0.663666,43.370375,-8.422900,1,-0.010575,-0.042373,4.116002


In [34]:
import os

output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, 'processed_data.csv')
df.to_csv(output_file, index=False)